In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from libs.ds_charts import get_variable_types
import libs.ds_charts as ds
import datetime

ModuleNotFoundError: No module named 'config_prof'

# Missing values imputation

In [ ]:
dataset_1 = pd.read_csv('data/NYC_collisions_tabular.csv', na_values='NaN')
new_dataset_1 = dataset_1.copy()

#FIND VARIABLES WITH MISSING VALUES
mv = {}
for var in new_dataset_1:
    nr = new_dataset_1[var].isna().sum()
    if nr > 0:
        mv[var] = nr

#DISCARD COLUMNS WITH MORE THEN 90% MISSING VALUES
threshold = new_dataset_1.shape[0] * 0.85

missings = [c for c in mv.keys() if mv[c]>threshold]
new_dataset_1.drop(columns=missings, inplace=True)
print('Dropped variables', missings)

#DISCARD RECORDS WITH MAJORITY OF MISSING VALUES
threshold = new_dataset_1.shape[1] * 0.50

new_dataset_1.dropna(thresh=threshold, inplace=True)
print(new_dataset_1.shape)

#PERSON_AGE
person_age = dataset_1['PERSON_AGE']
mean_ages = int(person_age.mean())
new_dataset_1['PERSON_AGE'].fillna(mean_ages,inplace=True)

#SAFETY_EQUIPMENT
new_dataset_1['SAFETY_EQUIPMENT'].fillna('Unknown',inplace=True)

#EJECTION
new_dataset_1['EJECTION'].fillna('Not Ejected',inplace=True)

#VEHICLE_ID
new_dataset_1['VEHICLE_ID'].dropna(inplace=True)

#POSITION IN VEHICLE
new_dataset_1['POSITION_IN_VEHICLE'].fillna('Unknown',inplace=True)
new_dataset_1.dropna(inplace=True)


### Dummification

In [ ]:
file = 'nyc_collisions'
filename = 'data/nyc_collisions.csv'
week_days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

### weekdays
new_dataset_1['CRASH_DATE'] = new_dataset_1['CRASH_DATE'].apply(lambda x: week_days[datetime.date(int(x.split('/')[0]),int(x.split('/')[1]),int(x.split('/')[0])).weekday()])  

symbolic_vars = ['CRASH_TIME','CRASH_DATE','BODILY_INJURY','SAFETY_EQUIPMENT','PERSON_SEX','EJECTION','PERSON_TYPE','COMPLAINT','EMOTIONAL_STATUS','POSITION_IN_VEHICLE','PED_ROLE']

## Transform hours to 0-22
new_dataset_1['CRASH_TIME'] = new_dataset_1['CRASH_TIME'].apply(lambda x: int(x.split(':')[0]))

print(new_dataset_1.columns)

def dummify(df, vars_to_dummify):
    other_vars = [c for c in df.columns if not c in vars_to_dummify]
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype=bool)
    X = df[vars_to_dummify]
    encoder.fit(X)
    new_vars = encoder.get_feature_names_out(vars_to_dummify)
    trans_X = encoder.transform(X)
    dummy = pd.DataFrame(trans_X, columns=new_vars, index=X.index)
    dummy = dummy.convert_dtypes(convert_boolean=True)

    final_df = pd.concat([df[other_vars], dummy], axis=1)
    return final_df

variables = get_variable_types(new_dataset_1)
new_dataset_1 = dummify(new_dataset_1, symbolic_vars)

#remove ids, 
new_dataset_1.drop('PERSON_ID',axis=1,inplace=True)
new_dataset_1.drop('UNIQUE_ID',axis=1,inplace=True)
new_dataset_1.drop('COLLISION_ID',axis=1,inplace=True)
new_dataset_1.drop('VEHICLE_ID',axis=1,inplace=True)
new_dataset_1.drop('BODILY_INJURY_Unknown',axis=1,inplace=True)
new_dataset_1.drop('SAFETY_EQUIPMENT_None',axis=1,inplace=True)
new_dataset_1.drop('SAFETY_EQUIPMENT_Unknown',axis=1,inplace=True)
new_dataset_1.drop('COMPLAINT_Unknown',axis=1,inplace=True)
new_dataset_1.drop('EMOTIONAL_STATUS_Unknown',axis=1,inplace=True)
new_dataset_1.drop('POSITION_IN_VEHICLE_Unknown',axis=1,inplace=True)



new_dataset_1.to_csv(f'data/{file}_dummified.csv', index=False)
nr = new_dataset_1.isna().sum()

print(new_dataset_1.columns)

### Outliers imputation

In [ ]:
# AGE
new_dataset_1 = new_dataset_1[(new_dataset_1.PERSON_AGE < 110) & (new_dataset_1.PERSON_AGE > 0)]

## Scalling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from matplotlib.pyplot import subplots, show

'''
 we need to apply the transfomation and rejoin the data together in order
 to have a unique dataframe. They can only be applied to numerical data, 
 without any missing value. In order to do that, we are splitting our dataframe into three 
 dataframes, one for each data type: above, discarding date variables, since the majority of 
 techniques are not able to deal with them.
'''

variable_types = get_variable_types(new_dataset_1)
numeric_vars =  variable_types['Numeric'] # ['PERSON_AGE'] 
symbolic_vars2 = variable_types['Symbolic'] # ['CRASH_DATE, CRASH_TIME]
boolean_vars = variable_types['Binary'] # all the symbolic are binary now!

df_nr = new_dataset_1[numeric_vars]
df_sb = new_dataset_1[symbolic_vars2]
df_bool = new_dataset_1[boolean_vars]

#print(numeric_vars)
#print(symbolic_vars2)
#print(boolean_vars)

In [ ]:
transf = StandardScaler(with_mean=True, with_std=True, copy=True).fit(df_nr)
tmp = pd.DataFrame(transf.transform(df_nr), index=new_dataset_1.index, columns= numeric_vars)
norm_data_zscore = pd.concat([tmp, df_sb,  df_bool], axis=1)
norm_data_zscore.to_csv(f'data/{file}_scaled_zscore.csv', index=False)

transf = MinMaxScaler(feature_range=(0, 1), copy=True).fit(df_nr)
tmp = pd.DataFrame(transf.transform(df_nr), index=new_dataset_1.index, columns= numeric_vars)
norm_data_minmax = pd.concat([tmp, df_sb,  df_bool], axis=1)
norm_data_minmax.to_csv(f'data/{file}_scaled_minmax.csv', index=False)
print(norm_data_minmax.describe())
df = new_dataset_1
# Now we can se the result of the transformed data with a single boxplot, again.

fig, axs = subplots(1, 3, figsize=(20,10),squeeze=False)
axs[0, 0].set_title('Original data')
df.boxplot(ax=axs[0, 0])
axs[0, 1].set_title('Z-score normalization')
norm_data_zscore.boxplot(ax=axs[0, 1])
axs[0, 2].set_title('MinMax normalization')
norm_data_minmax.boxplot(ax=axs[0, 2])
show()

### TRAIN AND TEST ZSCORE

In [ ]:
import numpy as np
from pandas import read_csv, concat, unique, DataFrame
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

file_tag = 'nyc_collisions_scaled_zscore'
data: DataFrame = read_csv('data/nyc_collisions_scaled_zscore.csv')
target = 'PERSON_INJURY'
positive = 'Injured'
negative = 'Killed'
values = {'Original': [len(data[data[target] == positive]), len(data[data[target] == negative])]}

y: np.ndarray = data.pop(target).values
X: np.ndarray = data.values
labels: np.ndarray = unique(y)
labels.sort()

trnX, tstX, trnY, tstY = train_test_split(X, y, train_size=0.7, stratify=y)

train = concat([DataFrame(trnX, columns=data.columns), DataFrame(trnY,columns=[target])], axis=1)
train.to_csv(f'data/{file_tag}_train.csv', index=False)

test = concat([DataFrame(tstX, columns=data.columns), DataFrame(tstY,columns=[target])], axis=1)
test.to_csv(f'data/{file_tag}_test.csv', index=False)
values['Train'] = [len(np.delete(trnY, np.argwhere(trnY==negative))), len(np.delete(trnY, np.argwhere(trnY==positive)))]
values['Test'] = [len(np.delete(tstY, np.argwhere(tstY==negative))), len(np.delete(tstY, np.argwhere(tstY==positive)))]

plt.figure(figsize=(12,4))
ds.multiple_bar_chart([positive, negative], values, title='Data distribution per dataset')
plt.show()

### TRAIN AND TEST MINMAX

In [ ]:
import numpy as np
from pandas import read_csv, concat, unique, DataFrame
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

file_tag = 'nyc_collisions_scaled_minmax'
data: DataFrame = read_csv('data/nyc_collisions_scaled_minmax.csv')
target = 'PERSON_INJURY'
positive = 'Injured'
negative = 'Killed'
values = {'Original': [len(data[data[target] == positive]), len(data[data[target] == negative])]}

y: np.ndarray = data.pop(target).values
X: np.ndarray = data.values
labels: np.ndarray = unique(y)
labels.sort()

trnX, tstX, trnY, tstY = train_test_split(X, y, train_size=0.7, stratify=y)

train = concat([DataFrame(trnX, columns=data.columns), DataFrame(trnY,columns=[target])], axis=1)
train.to_csv(f'data/{file_tag}_train.csv', index=False)

test = concat([DataFrame(tstX, columns=data.columns), DataFrame(tstY,columns=[target])], axis=1)
test.to_csv(f'data/{file_tag}_test.csv', index=False)
values['Train'] = [len(np.delete(trnY, np.argwhere(trnY==negative))), len(np.delete(trnY, np.argwhere(trnY==positive)))]
values['Test'] = [len(np.delete(tstY, np.argwhere(tstY==negative))), len(np.delete(tstY, np.argwhere(tstY==positive)))]

plt.figure(figsize=(12,4))
ds.multiple_bar_chart([positive, negative], values, title='Data distribution per dataset')
plt.show()